In [12]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.instance.SAT_Instance import SAT_Instance, SAT_from_index_file
from src.solver.SAT_Riss_Solver import SAT_Riss_Solver
from src.constant import DATA_DIR

In [20]:
with open("minisat1m_yes.uri") as f:
    lines = f.readlines()
lines = [line.strip() for line in lines if line.strip()]

dir_ = DATA_DIR / "SAT" / "gbd"

In [22]:
lines

['http://benchmark-database.de/file/000a41cdca43be89ed62ea3abf2d0b64',
 'http://benchmark-database.de/file/0033baadbd48d3f880e9b5a733bd4950',
 'http://benchmark-database.de/file/003a77d2aa15a5f93aa2cfe79b986c9e',
 'http://benchmark-database.de/file/005ccb378ced61c02105ed7ee0a62038',
 'http://benchmark-database.de/file/0064aa2e16c9d4b18d32a45d3073fce9',
 'http://benchmark-database.de/file/00a981f385cd209d41e1efa1f5f29bac',
 'http://benchmark-database.de/file/00b9b467c26540853360491401da61a5',
 'http://benchmark-database.de/file/00be590675417eba2bb2585790ac392d',
 'http://benchmark-database.de/file/00e289099ea9bcef6a056a8c8ba8a3c0',
 'http://benchmark-database.de/file/00f2eb377986e7decbc863931680a3b2',
 'http://benchmark-database.de/file/00f969737ba4338bd233cd3ed249bd55',
 'http://benchmark-database.de/file/010c9d7a6e9e6cac63f110915d9dd56e',
 'http://benchmark-database.de/file/0141b82b3d2a411d8c2168119e2d7452',
 'http://benchmark-database.de/file/0152d286cde406747d179c36a95c2fdc',
 'http

In [8]:
solver = SAT_Riss_Solver()
instances = SAT_from_index_file(filepath=DATA_DIR / "SAT" / "index_u150.json", max_cost=10.0, max_time=10.0)


In [11]:
instances

InstanceList(size=10)[SAT_Instance(filepath=SAT/uf150-01.cnf), SAT_Instance(filepath=SAT/uf150-02.cnf), SAT_Instance(filepath=SAT/uf150-03.cnf), SAT_Instance(filepath=SAT/uf150-04.cnf), SAT_Instance(filepath=SAT/uf150-05.cnf), SAT_Instance(filepath=SAT/uuf150-01.cnf), SAT_Instance(filepath=SAT/uuf150-02.cnf), SAT_Instance(filepath=SAT/uuf150-03.cnf), SAT_Instance(filepath=SAT/uuf150-04.cnf), SAT_Instance(filepath=SAT/uuf150-05.cnf)]

In [5]:
solver = SAT_Riss_Solver()
instances = SAT_from_index_file(filepath=DATA_DIR / "SAT" / "index_u75.json", max_cost=10.0, max_time=10.0)

for instance in instances:
    solver.solve(instance, prefix="test")

[2025-05-27 12:56:35] DEBUG     solve(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uf75-01.cnf))
[2025-05-27 12:56:35] DEBUG     Solver.Result(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uf75-01.cnf), cost=10.0000, time=10.0000, cached=False, surrogate=False, error=True)
[2025-05-27 12:56:35] DEBUG     solve(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uf75-02.cnf))
[2025-05-27 12:56:35] DEBUG     Solver.Result(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uf75-02.cnf), cost=10.0000, time=10.0000, cached=False, surrogate=False, error=True)
[2025-05-27 12:56:35] DEBUG     solve(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uf75-03.cnf))
[2025-05-27 12:56:35] DEBUG     Solver.Result(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uf75-03.cnf), cost=10.0000

Error running solver=Solver(id=80961051775322991) on instance=SAT_Instance(filepath=SAT/uf75-01.cnf): [WinError 2] The system cannot find the file specified
Error running solver=Solver(id=80961051775322991) on instance=SAT_Instance(filepath=SAT/uf75-02.cnf): [WinError 2] The system cannot find the file specified
Error running solver=Solver(id=80961051775322991) on instance=SAT_Instance(filepath=SAT/uf75-03.cnf): [WinError 2] The system cannot find the file specified
Error running solver=Solver(id=80961051775322991) on instance=SAT_Instance(filepath=SAT/uf75-04.cnf): [WinError 2] The system cannot find the file specified


[2025-05-27 12:56:35] DEBUG     Solver.Result(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uf75-05.cnf), cost=10.0000, time=10.0000, cached=False, surrogate=False, error=True)
[2025-05-27 12:56:35] DEBUG     solve(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uuf75-01.cnf))
[2025-05-27 12:56:35] DEBUG     Solver.Result(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uuf75-01.cnf), cost=10.0000, time=10.0000, cached=False, surrogate=False, error=True)
[2025-05-27 12:56:35] DEBUG     solve(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uuf75-02.cnf))
[2025-05-27 12:56:35] DEBUG     Solver.Result(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uuf75-02.cnf), cost=10.0000, time=10.0000, cached=False, surrogate=False, error=True)
[2025-05-27 12:56:35] DEBUG     solve(prefix=test, solver=Solver(id=809610

Error running solver=Solver(id=80961051775322991) on instance=SAT_Instance(filepath=SAT/uf75-05.cnf): [WinError 2] The system cannot find the file specified
Error running solver=Solver(id=80961051775322991) on instance=SAT_Instance(filepath=SAT/uuf75-01.cnf): [WinError 2] The system cannot find the file specified
Error running solver=Solver(id=80961051775322991) on instance=SAT_Instance(filepath=SAT/uuf75-02.cnf): [WinError 2] The system cannot find the file specified
Error running solver=Solver(id=80961051775322991) on instance=SAT_Instance(filepath=SAT/uuf75-03.cnf): [WinError 2] The system cannot find the file specified
Error running solver=Solver(id=80961051775322991) on instance=SAT_Instance(filepath=SAT/uuf75-04.cnf): [WinError 2] The system cannot find the file specified


[2025-05-27 12:56:35] DEBUG     solve(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uuf75-05.cnf))
[2025-05-27 12:56:35] DEBUG     Solver.Result(prefix=test, solver=Solver(id=80961051775322991), instance=SAT_Instance(filepath=SAT/uuf75-05.cnf), cost=10.0000, time=10.0000, cached=False, surrogate=False, error=True)


Error running solver=Solver(id=80961051775322991) on instance=SAT_Instance(filepath=SAT/uuf75-05.cnf): [WinError 2] The system cannot find the file specified


In [6]:
for instance in instances:
    print(instance)

SAT_Instance(filepath=SAT/uf250-01.cnf)
SAT_Instance(filepath=SAT/uf250-02.cnf)
SAT_Instance(filepath=SAT/uf250-03.cnf)
SAT_Instance(filepath=SAT/uf250-04.cnf)
SAT_Instance(filepath=SAT/uf250-05.cnf)
SAT_Instance(filepath=SAT/uuf250-01.cnf)
SAT_Instance(filepath=SAT/uuf250-02.cnf)
SAT_Instance(filepath=SAT/uuf250-03.cnf)
SAT_Instance(filepath=SAT/uuf250-04.cnf)
SAT_Instance(filepath=SAT/uuf250-05.cnf)


In [ ]:
solver.to_dict()

In [ ]:
test_instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "TEST_400" / "index.json", max_cost=100.0, max_time=10.0)
instance = test_instances[23]
instance

In [ ]:
solver = TSP_LKH_Solver()
solver

In [ ]:
solver.solve(instance, prefix="test")

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(4.5, 4.5))

df = instance._read_file_to_df()
plt.scatter(df["X"], df["Y"], s=6)
ax.ticklabel_format(style="scientific", axis="both", scilimits=(0, 0))
plt.grid(color="black", alpha=0.05)

plt.xlabel("X coordinate")
plt.ylabel("Y coordinate")
plt.savefig("grid.pdf", bbox_inches="tight", transparent=True)
plt.show()

In [ ]:
idx = []
with open("out") as f:
    for line in f:
        if line.startswith("NAME"):
            continue
        elif line.startswith("COMMENT"):
            continue
        elif line.startswith("TYPE"):
            continue
        elif line.startswith("DIMENSION"):
            continue
        elif line.startswith("TOUR"):
            continue
        elif line.startswith("-1"):
            break
        idx.append(int(line.strip()))

fig, ax = plt.subplots(figsize=(4.5, 4.5))
df = instance._read_file_to_df()
plt.plot(df.loc[idx, "X"], df.loc[idx, "Y"], c="red", lw=0.8, zorder=1)
plt.scatter(df["X"], df["Y"], s=6, zorder=2)
ax.ticklabel_format(style="scientific", axis="both", scilimits=(0, 0))
plt.grid(color="black", alpha=0.05)
plt.xlabel("X coordinate")
plt.ylabel("Y coordinate")
plt.savefig("grid_solved.pdf", bbox_inches="tight", transparent=True)
plt.show()

In [ ]:
instance = test_instances[4]
fig, ax = plt.subplots(figsize=(4.5, 4.5))

df = instance._read_file_to_df()
plt.scatter(df["X"], df["Y"], s=6)
ax.ticklabel_format(style="scientific", axis="both", scilimits=(0, 0))
plt.grid(color="black", alpha=0.05)

plt.xlabel("X coordinate")
plt.ylabel("Y coordinate")
plt.savefig("cluster_netgen.pdf", bbox_inches="tight", transparent=True)
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 4.5))

df = instance._read_file_to_df()
plt.scatter(df["X"], df["Y"], s=6)
ax.ticklabel_format(style="scientific", axis="both", scilimits=(0, 0))
plt.grid(color="black", alpha=0.05)

plt.xlabel("X coordinate")
plt.ylabel("Y coordinate")
plt.savefig("grid.pdf", bbox_inches="tight", transparent=True)
plt.show()

In [ ]:
# import joblib
# paths = list((MAIN_DIR / "_archive" / "experiments" / "2025-01-27" / "200" / "database").glob("*.db"))
# for path in paths:
#     db = DB(path)

#     configured_portfolio_solvers = pd.read_sql_query(""" 
#     SELECT DISTINCT solver_id FROM results
#     WHERE prefix LIKE 'test%'                  
#     """, db._conn)["solver_id"].to_list()

#     portfolio = Portfolio.from_iterable([TSP_LKH_Solver.from_db(id_, db) for id_ in configured_portfolio_solvers])
#     joblib.dump(portfolio, MAIN_DIR / "portfolios" / "200" / f"{path.stem}.pkl")

# paths = list((MAIN_DIR / "_archive" / "experiments" / "2025-01-27" / "400" / "database").glob("*.db"))
# for path in paths:
#     db = DB(path)

#     configured_portfolio_solvers = pd.read_sql_query(""" 
#     SELECT DISTINCT solver_id FROM results
#     WHERE prefix LIKE 'test%'                  
#     """, db._conn)["solver_id"].to_list()

#     portfolio = Portfolio.from_iterable([TSP_LKH_Solver.from_db(id_, db) for id_ in configured_portfolio_solvers])
#     joblib.dump(portfolio, MAIN_DIR / "portfolios" / "400" / f"{path.stem}.pkl")

In [ ]:
# import joblib

# paths = list((MAIN_DIR / "portfolios" / "200").glob("run-plain-30-*.pkl"))
# for path in paths:
#     run_id = path.stem.split("-")[-1]
#     portfolio = joblib.load(path)

#     for i in range(100):
#         portfolio.evaluate(
#             test_instances,
#             prefix=f"{run_id};test{i}",
#             calculate_features=False,
#             cache=False,
#         )